In [1]:
import numpy as np
import random
import pandas 
import pickle
import os
import time
from preprocessing_dailydialogue import *

In [2]:
with open('../data/ecm/ecm_data1.pickle','rb') as f:
    pairs = pickle.load(f)
    pairs_emotion = pickle.load(f)
    
with open('../data/ecm/ecm_data2.pickle','rb') as f:#
    pairs2 = pickle.load(f)
    pairs_emotion2 = pickle.load(f)

In [3]:
pairs = pairs + pairs2
pairs_emotion = pairs_emotion + pairs_emotion2

In [4]:
len(pairs)

313421

In [5]:
index = range(len(pairs))
index_train = random.sample(index,len(pairs) - 5000)
index = set(index) - set(index_train)
index_test = random.sample(index,5000)
index_val = list(set(index) - set(index_test))

In [6]:
train =[]
train_emotion =[]
for each in index_train:
    train.append(pairs[each])
    train_emotion.append(pairs_emotion[each])
    
test =[]
test_emotion =[]
for each in index_test:
    test.append(pairs[each])
    test_emotion.append(pairs_emotion[each])
    
val =[]
val_emotion =[]
for each in index_val:
    val.append(pairs[each])
    val_emotion.append(pairs_emotion[each])

In [7]:
with open('train.pickle','wb') as f:
    pickle.dump(train,f)
    pickle.dump(train_emotion,f)

In [8]:
with open('test.pickle','wb') as f:
    pickle.dump(test,f)
    pickle.dump(test_emotion,f)

In [9]:
with open('val.pickle','wb') as f:
    pickle.dump(val,f)
    pickle.dump(val_emotion,f)

In [10]:
with open('val.pickle','rb') as f:
    a = pickle.load(f)
    b = pickle.load(f)

In [7]:
len(train)

308421

# Preprocessing

In [8]:
# load from begining .
# Default word tokens
#
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
MAX_LENGTH = 10  # Maximum sentence length to consider
MIN_COUNT = 3    # Minimum word count threshold for trimming
save_dir = os.path.join("data", "save")
emo_dict = { 0: 'neutral', 1: 'joy', 2: 'anger', 
            3: 'sadness',4:'fear'}
emo2idx = {value:key for key,value in emo_dict.items()}

with open('train.pickle','rb') as f:
    pairs = pickle.load(f)
    pairs_emotion = pickle.load(f)
print('Loading data',len(pairs),len(pairs_emotion))
# subset the training for testing 
MIN_LENGTH = 5 # min length of sentence to keep
def process(pairs,pairs_emotion,voc = None,emo2idx = emo2idx):
    pairs_emotion_index = []
    for each in pairs_emotion:
        pairs_emotion_index.append([emo2idx[each[0]],emo2idx[each[1]]])
    pairs_emotion = pairs_emotion_index
    del pairs_emotion_index
    
    if voc is None:
        count = 0
        voc = Voc(name='train',max_length=MAX_LENGTH,min_count=MIN_COUNT)
        pairs,pairs_emotion = filterPairs(pairs,pairs_emotion,voc.max_length,MIN_LENGTH)
        print(len(pairs),len(pairs_emotion))
        for each in pairs:
            count += 1
            voc.addSentence(each[0])
            voc.addSentence(each[1])
    else:
        pairs,pairs_emotion = filterPairs(pairs,pairs_emotion,voc.max_length,MIN_LENGTH)
    pairs,pairs_emotion = trimRareWords(pairs=pairs,pairs_emotion=pairs_emotion,voc=voc,min_count=voc.min_count)
    print(len(pairs),len(pairs_emotion))
    
    return pairs,pairs_emotion,voc



Loading data 308421 308421


In [9]:
len(pairs),len(pairs_emotion)

(308421, 308421)

In [10]:
pairs,pairs_emotion,voc = process(pairs,pairs_emotion)

36724 36724
keep_words 5966 / 14458 = 0.4126
Trimmed from 36724 pairs to 28281, 0.7701 of total
28281 28281


In [13]:
for sentence1,sentence2 in pairs:
    voc.addSentence(sentence1)
    voc.addSentence(sentence2)

In [14]:
voc.word2count

{'PAD': 3,
 'SOS': 3,
 'EOS': 3,
 'i': 16231,
 've': 741,
 'tried': 52,
 'it': 7492,
 'not': 1978,
 'a': 5627,
 'chance': 50,
 '.': 47767,
 'seems': 55,
 'you': 18664,
 'have': 2764,
 'to': 6357,
 'take': 672,
 'time': 664,
 'do': 4369,
 'want': 1425,
 'drink': 183,
 '?': 20504,
 'yeah': 981,
 'right': 1109,
 'm': 2697,
 'junior': 13,
 'no': 2470,
 'mean': 637,
 'in': 2097,
 'your': 2340,
 'pregnancy': 4,
 'hey': 307,
 'is': 3773,
 'chandler': 10,
 'here': 1678,
 'he': 2014,
 's': 7116,
 'oh': 1146,
 'we': 2363,
 're': 1714,
 'doing': 602,
 'about': 1654,
 'thirty': 87,
 'pretty': 164,
 'good': 931,
 'u': 336,
 'miles': 30,
 'an': 459,
 'hour': 65,
 'met': 66,
 'before': 219,
 'don': 2653,
 't': 5540,
 'think': 1214,
 'so': 1358,
 'really': 752,
 'need': 689,
 'go': 1293,
 'shopping': 38,
 'what': 6425,
 'buy': 148,
 'would': 1042,
 'suggest': 13,
 'see': 911,
 'know': 2001,
 '.it': 27,
 'depends': 38,
 'pay': 156,
 'for': 2159,
 'electricity': 14,
 'myself': 69,
 'yes': 1707,
 'includ

In [19]:
len(test),len(test_emotion)

(5000, 5000)

In [17]:
pairs_val,pairs_val_emotion,voc = process(test,test_emotion) 

637 637
keep_words 350 / 1480 = 0.2365
Trimmed from 637 pairs to 91, 0.1429 of total
91 91


In [18]:
len(pairs_val),len(pairs_val_emotion)

(91, 91)

In [43]:
voc.index2word[1450]

'water'

In [44]:
voc.num_words

15328

In [15]:
set(voc.word2count.values())

{1, 3}

In [19]:
with open('test.pickle','rb') as f:
    pairs_t = pickle.load(f)
    pairs_emotion_t = pickle.load(f)

In [8]:
len(pairs_t),len(pairs_emotion_t)

NameError: name 'pairs_t' is not defined

In [7]:
pairs_t,pairs_emotion_t,voc_t = process(pairs_t,pairs_emotion_t,voc)

NameError: name 'pairs_t' is not defined

In [17]:
print(pairs[:5],pairs_emotion[:5])
print(pairs_t[:5],pairs_emotion_t[:5])

[['i m still here .', 'well then talk to me some more .'], ['ben i m afraid i ve got to ask', 'not unless you want that clipboard stretched up your'], ['shit we didn t sign in .', 'so get the hell out of here .'], ['you mean this one ?', 'no the other one in the brown case .'], ['get the outside shutters .', 'what are we doing ?']] [[4, 2], [1, 3], [3, 4], [0, 0], [3, 4]]


NameError: name 'pairs_t' is not defined

In [22]:
len(pairs_t),len(pairs_emotion_t)

(3012, 3012)

In [23]:
for each in pairs_t:
    if len(each[0].split(' ')) >= voc.max_length:
        print(each)

In [18]:
with open('processed_train_large.pickle','wb') as f:
    pickle.dump(pairs,f)
    pickle.dump(pairs_emotion,f)
    pickle.dump(voc,f)
    
#with open('processed_test.pickle','wb') as f:
#    pickle.dump(pairs_t,f)
#    pickle.dump(pairs_emotion_t,f)

In [10]:
len(pairs),len(pairs_emotion)

(214361, 214361)